# 제주도 수온 데이터

In [40]:
import pandas as pd

import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time

<br><hr>

### 00. 월별 평균 수온 데이터 크롤링

In [26]:
# 지역별 URL 매핑
locations = {
    '제주': 'DT_0004',
    '서제주': 'seojeju',
    '모슬포': 'DT_0023',
    '중문': 'TW_0075',
    '서귀포': 'DT_0010',
    '성산포': 'DT_0022',
    '추자도': 'DT_0021'
}

In [27]:
# 데이터 수집을 위한 반복
data = []
for location, param in locations.items():
    for month in range(1, 13):
        url = f'https://m.badatime.com/view_temp_pa.jsp?idx=77&param={param}&date1=2023-{month:02d}'
        response = requests.get(url)
        
        # 페이지 로드 시간을 기다림
        time.sleep(1)
        
        soup = BeautifulSoup(response.content, 'html.parser')

        # HTML 태그 구조에 맞춰 데이터 추출
        rows = soup.find_all('tr', bgcolor=['#F6F6F6', '#ffffff'])  # 온도 데이터가 포함된 행 찾기

        for row in rows:
            cols = row.find_all('td')
            if len(cols) >= 5:  # 날짜, 최저 시간, 최저 온도, 최고 시간, 최고 온도 존재 시
                try:
                    date_str = cols[0].text.strip()
                    date = datetime.strptime(f'2023-{date_str[:2]}-{date_str[4:6]}', '%Y-%m-%d')
                    low_temp = float(cols[2].text.strip().replace('℃', ''))
                    high_temp = float(cols[4].text.strip().replace('℃', ''))
                    data.append({'수온측정위치': location, 'date': date, 'low_temp': low_temp, 'high_temp': high_temp})
                except ValueError:
                    continue  # 날짜 형식 또는 온도 데이터가 잘못된 경우 무시

In [28]:
# 데이터프레임 생성
if data:
    df = pd.DataFrame(data)

    # 월별 평균 계산
    df['month'] = df['date'].dt.strftime('%m')  # 01~12 형식으로 변경
    monthly_avg = df.groupby(['수온측정위치', 'month']).agg(
        avg_low_temp=('low_temp', 'mean'),
        avg_high_temp=('high_temp', 'mean')
    ).reset_index()

    # 소수 둘째 자리까지 출력 형식 지정
    monthly_avg['avg_low_temp'] = monthly_avg['avg_low_temp'].round(2)
    monthly_avg['avg_high_temp'] = monthly_avg['avg_high_temp'].round(2)

else:
    print("데이터를 찾을 수 없습니다.")

<br><hr>

### 01. 월별 최고기온을 가진 지역 확인하기

In [29]:
# 월별 avg_low_temp 중 최고 기온을 가진 지역과 월, avg_low_temp를 출력
max_avg_low_temp = monthly_avg.loc[monthly_avg.groupby('month')['avg_low_temp'].idxmax()]

# 필요한 열만 선택하여 출력
result = max_avg_low_temp[['수온측정위치', 'month', 'avg_low_temp']]

# 결과 출력
result

,수온측정위치,month,avg_low_temp
12,서귀포,01,16.79
13,서귀포,02,16.20
14,서귀포,03,16.45
15,서귀포,04,17.50
16,서귀포,05,18.12
65,중문,06,20.31
54,제주,07,24.05
67,중문,08,28.39
68,중문,09,26.82
69,중문,10,23.75


In [30]:
# 월별 avg_high_temp 중 최고 기온을 가진 지역과 월, avg_high_temp를 출력
max_avg_high_temp = monthly_avg.loc[monthly_avg.groupby('month')['avg_high_temp'].idxmax()]

# 필요한 열만 선택하여 출력
result = max_avg_high_temp[['수온측정위치', 'month', 'avg_high_temp']]

# 결과 출력
result

,수온측정위치,month,avg_high_temp
12,서귀포,01,17.32
13,서귀포,02,16.53
14,서귀포,03,16.80
15,서귀포,04,17.89
16,서귀포,05,18.55
65,중문,06,21.61
66,중문,07,25.08
67,중문,08,29.35
68,중문,09,27.62
21,서귀포,10,24.55


In [31]:
monthly_avg['수온측정위치'].value_counts()

수온측정위치
모슬포    12
서귀포    12
서제주    12
성산포    12
제주     12
중문     12
추자도    12
Name: count, dtype: int64

In [33]:
# 칼럼명 변경
df.rename(columns={'제주도 해수욕장(물놀이)': '해수욕장'}, inplace=True)

<br><hr>

### 02. 해수욕장별 특징 추출

In [41]:
import re
import json
from selenium import webdriver

In [125]:
# Selenium WebDriver 설정
driver = webdriver.Chrome()

In [117]:
# URL 목록 정의
urls = {
    '하모 해수욕장': 'https://pcmap.place.naver.com/place/13491319/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202411110042',
    '하효쇠소깍 해수욕장': 'https://pcmap.place.naver.com/place/36072556/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202411110042',
    '사계 해수욕장': 'https://pcmap.place.naver.com/place/17083214/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202411110019',
    '자구리공원 담수욕장': 'https://pcmap.place.naver.com/place/1196341713/home?entry=pll&from=nx&fromNxList=true&from=map&fromPanelNum=2&timestamp=202411110022',
    '협재 해수욕장': 'https://pcmap.place.naver.com/place/11491807/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202411110022',
    '금능 해수욕장': 'https://pcmap.place.naver.com/place/13491073/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202411110023',
    '산호 해수욕장': 'https://pcmap.place.naver.com/place/13491787/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202411110023',
    '하고수동 해수욕장': 'https://pcmap.place.naver.com/place/13491867/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202411110024',
    '검멀레 해수욕장': 'https://pcmap.place.naver.com/place/13491583/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202411110024',
    '세화 해수욕장': 'https://pcmap.place.naver.com/place/13491684/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202411110029',
    '하도 해수욕장': 'https://pcmap.place.naver.com/place/13491451/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202411110031',
    '신양섭지 해수욕장': 'https://pcmap.place.naver.com/place/11491743/home?from=map&fromPanelNum=1&additionalHeight=76&timestamp=202411110032',
    '함덕 해수욕장': 'https://pcmap.place.naver.com/place/11491805/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202411110032',
    '이호테우 해수욕장': 'https://pcmap.place.naver.com/place/11491767/home?from=map&fromPanelNum=1&additionalHeight=76&timestamp=202411110033',
    '삼양 해수욕장': 'https://pcmap.place.naver.com/place/13491607/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202411110033',
    '신흥리 해수욕장': 'https://pcmap.place.naver.com/place/1310891704/home?from=map&fromPanelNum=1&additionalHeight=76&timestamp=202411110041',
    '월정리 해수욕장': 'https://pcmap.place.naver.com/place/20561777/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202411110038',
    '김녕 해수욕장': 'https://pcmap.place.naver.com/place/13491485/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202411110038',
    '중문색달 해수욕장': 'https://pcmap.place.naver.com/place/11491786/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202411110039',
    '화순 금모래 해수욕장': 'https://pcmap.place.naver.com/place/1081774025/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202411110039'
}
# 추자도 (후포해변, 모진이 몽돌해변) 데이터 없음

In [128]:
# 데이터 저장 리스트
results = []

# 각 URL에 접속하여 데이터 수집
for place_name, url in urls.items():
    driver.get(url)
    time.sleep(5)  # 페이지 로딩 대기

    # 페이지 소스 가져오기
    page_source = driver.page_source
    
    # JSON 데이터 추출
    pattern = r'{"__typename":"VisitorReviewStatsAnalysisVoteKeywordDetail".*?}'
    matches = re.findall(pattern, page_source)
    
    for match in matches:
        try:
            # JSON 객체로 변환
            data = json.loads(match)
            display_name = data.get('displayName')
            count = data.get('count')
            
            # 데이터 저장
            if display_name and count is not None:
                results.append({'해수욕장': place_name, 'review': display_name, 'Count': count})
        
        except json.JSONDecodeError:
            print(f"Failed to parse JSON for {place_name}")
    print(f'저장 완료 {place_name}')

# 데이터프레임 생성
df_review = pd.DataFrame(results)

# 브라우저 종료
driver.quit()

저장 완료 하모 해수욕장
저장 완료 하효쇠소깍 해수욕장
저장 완료 사계 해수욕장
저장 완료 자구리공원 담수욕장
저장 완료 협재 해수욕장
저장 완료 금능 해수욕장
저장 완료 산호 해수욕장
저장 완료 하고수동 해수욕장
저장 완료 검멀레 해수욕장
저장 완료 세화 해수욕장
저장 완료 하도 해수욕장
저장 완료 신양섭지 해수욕장
저장 완료 함덕 해수욕장
저장 완료 이호테우 해수욕장
저장 완료 삼양 해수욕장
저장 완료 신흥리 해수욕장
저장 완료 월정리 해수욕장
저장 완료 김녕 해수욕장
저장 완료 중문색달 해수욕장
저장 완료 화순 금모래 해수욕장


In [135]:
df_review_over5 = df_review[df_review['Count'] >= 5].reset_index(drop=True)
df_review_over5

,해수욕장,review,Count
0,하모 해수욕장,조용히 쉬기 좋아요,7
1,하모 해수욕장,주차하기 편해요,6
2,하모 해수욕장,사진이 잘 나와요,6
3,하모 해수욕장,뷰가 좋아요,6
4,하모 해수욕장,경관이 독특해요,5
...,...,...,...
227,화순 금모래 해수욕장,조용히 쉬기 좋아요,13
228,화순 금모래 해수욕장,주차하기 편해요,12
229,화순 금모래 해수욕장,아이와 가기 좋아요,7
230,화순 금모래 해수욕장,사진이 잘 나와요,6


In [143]:
# place별 feature를 하나의 칼럼으로 만듦 (comma로 연결)
review_combined = df_review_over5.groupby('해수욕장')['review'].apply(lambda x: ', '.join(x)).reset_index()
review_combined

,해수욕장,review
0,검멀레 해수욕장,"경관이 독특해요, 사진이 잘 나와요, 뷰가 좋아요, 조용히 쉬기 좋아요"
1,금능 해수욕장,"물놀이하기 좋아요, 사진이 잘 나와요, 뷰가 좋아요, 모래가 고와요, 조용히 쉬기 ..."
2,김녕 해수욕장,"사진이 잘 나와요, 물놀이하기 좋아요, 모래가 고와요, 주차하기 편해요, 뷰가 좋아..."
3,사계 해수욕장,"경관이 독특해요, 사진이 잘 나와요, 뷰가 좋아요, 조용히 쉬기 좋아요, 주차하기 ..."
4,산호 해수욕장,"사진이 잘 나와요, 뷰가 좋아요, 물놀이하기 좋아요, 경관이 독특해요, 조용히 쉬기..."
5,삼양 해수욕장,"물놀이하기 좋아요, 모래가 고와요, 사진이 잘 나와요, 서핑하기 좋아요, 뷰가 좋아..."
6,세화 해수욕장,"사진이 잘 나와요, 물놀이하기 좋아요, 조용히 쉬기 좋아요, 뷰가 좋아요, 모래가 ..."
7,신양섭지 해수욕장,"사진이 잘 나와요, 주차하기 편해요, 조용히 쉬기 좋아요, 경관이 독특해요, 물놀이..."
8,신흥리 해수욕장,"물놀이하기 좋아요, 조용히 쉬기 좋아요, 사진이 잘 나와요, 모래가 고와요, 뷰가 좋아요"
9,월정리 해수욕장,"사진이 잘 나와요, 모래가 고와요, 물놀이하기 좋아요, 주차하기 편해요, 조용히 쉬..."


In [144]:
# 후포해변과 모진이 몽돌해변 추가
review_combined = pd.concat([review_combined, pd.DataFrame({
    '해수욕장': ['후포해변', '모진이 몽돌해변'],
    'review': ['파도에 부딪혀 동글동글해진 몽돌들이 있는 해변',
                '추자도이 하추자도에 위치한 모래없이 몽돌로만 이루어진 해수욕장. 몽돌은 제주도 말로 동글동글한 돌을 뜻하는데, 밀려가고 밀려오는 파도에 움직이는 몽돌소리가 주는 특유의 편안함이 있다. 여름에는 해수욕도 가능하다.']
})], ignore_index=True)

# 결과 출력
review_combined.shape

(22, 2)

<br><hr>

### 03. 해수욕장 데이터 + 제주도 수온 데이터 + 리뷰 데이터 병합

In [181]:
df = pd.read_csv('../data/jeju_beach_temp.csv')
df.shape

(22, 5)

In [183]:
# review 데이터프레임과 병합
jeju_beach_info = pd.merge(df, review_combined, left_on='해수욕장', right_on='해수욕장', how='left')

# 결과 출력
jeju_beach_info.head(3)

,수온측정위치,해수욕장,주소,위도,경도,review
0,성산포,검멀레 해수욕장,제주특별자치도 제주시 우도면 조일리,33.496762,126.968511,"경관이 독특해요, 사진이 잘 나와요, 뷰가 좋아요, 조용히 쉬기 좋아요"
1,서제주,금능 해수욕장,제주특별자치도 제주시 한림읍 협재리 2464,33.391738,126.236774,"물놀이하기 좋아요, 사진이 잘 나와요, 뷰가 좋아요, 모래가 고와요, 조용히 쉬기 ..."
2,제주,김녕 해수욕장,제주특별자치시도 제주시 구좌읍 김녕리,33.558513,126.759562,"사진이 잘 나와요, 물놀이하기 좋아요, 모래가 고와요, 주차하기 편해요, 뷰가 좋아..."


In [186]:
# 해수욕장 >> month >> 수온 측정 위치 순으로 정렬
jeju_beach_info = jeju_beach_info.sort_values(by=['수온측정위치', '해수욕장'])
jeju_beach_info = jeju_beach_info.reset_index(drop=True)
jeju_beach_info.head()

,수온측정위치,해수욕장,주소,위도,경도,review
0,모슬포,사계 해수욕장,제주특별자치도 서귀포시 안덕면 사계리 2294-37,33.231115,126.310399,"경관이 독특해요, 사진이 잘 나와요, 뷰가 좋아요, 조용히 쉬기 좋아요, 주차하기 ..."
1,모슬포,하모 해수욕장,제주특별자치도 서귀포시 대정읍 하모리,33.210924,126.260497,"조용히 쉬기 좋아요, 주차하기 편해요, 사진이 잘 나와요, 뷰가 좋아요, 경관이 독특해요"
2,서귀포,자구리공원 담수욕장,제주특별자치도 서귀포시 서귀동 70-1,33.243454,126.569093,"주차하기 편해요, 뷰가 좋아요"
3,서귀포,하효쇠소깍 해수욕장,제주특별자치도 서귀포시 하효동 995-5,33.251412,126.623452,"조용히 쉬기 좋아요, 경관이 독특해요, 주차하기 편해요, 사진이 잘 나와요, 뷰가 ..."
4,서제주,금능 해수욕장,제주특별자치도 제주시 한림읍 협재리 2464,33.391738,126.236774,"물놀이하기 좋아요, 사진이 잘 나와요, 뷰가 좋아요, 모래가 고와요, 조용히 쉬기 ..."


In [187]:
# monthly_avg 데이터프레임과 병합
jeju_beach_info_with_temp = pd.merge(jeju_beach_info, monthly_avg, left_on='수온측정위치', right_on='수온측정위치', how='left')

# 결과 출력
jeju_beach_info_with_temp

,수온측정위치,해수욕장,주소,위도,경도,review,month,avg_low_temp,avg_high_temp
0,모슬포,사계 해수욕장,제주특별자치도 서귀포시 안덕면 사계리 2294-37,33.231115,126.310399,"경관이 독특해요, 사진이 잘 나와요, 뷰가 좋아요, 조용히 쉬기 좋아요, 주차하기 ...",01,15.59,16.32
1,모슬포,사계 해수욕장,제주특별자치도 서귀포시 안덕면 사계리 2294-37,33.231115,126.310399,"경관이 독특해요, 사진이 잘 나와요, 뷰가 좋아요, 조용히 쉬기 좋아요, 주차하기 ...",02,15.03,15.67
2,모슬포,사계 해수욕장,제주특별자치도 서귀포시 안덕면 사계리 2294-37,33.231115,126.310399,"경관이 독특해요, 사진이 잘 나와요, 뷰가 좋아요, 조용히 쉬기 좋아요, 주차하기 ...",03,15.61,16.03
3,모슬포,사계 해수욕장,제주특별자치도 서귀포시 안덕면 사계리 2294-37,33.231115,126.310399,"경관이 독특해요, 사진이 잘 나와요, 뷰가 좋아요, 조용히 쉬기 좋아요, 주차하기 ...",04,16.94,17.35
4,모슬포,사계 해수욕장,제주특별자치도 서귀포시 안덕면 사계리 2294-37,33.231115,126.310399,"경관이 독특해요, 사진이 잘 나와요, 뷰가 좋아요, 조용히 쉬기 좋아요, 주차하기 ...",05,17.74,18.13
...,...,...,...,...,...,...,...,...,...
259,추자도,후포해변,제주특별자치도 제주시 추자면 대서리,33.962572,126.289335,파도에 부딪혀 동글동글해진 몽돌들이 있는 해변,08,23.12,24.21
260,추자도,후포해변,제주특별자치도 제주시 추자면 대서리,33.962572,126.289335,파도에 부딪혀 동글동글해진 몽돌들이 있는 해변,09,24.25,25.28
261,추자도,후포해변,제주특별자치도 제주시 추자면 대서리,33.962572,126.289335,파도에 부딪혀 동글동글해진 몽돌들이 있는 해변,10,19.99,20.34
262,추자도,후포해변,제주특별자치도 제주시 추자면 대서리,33.962572,126.289335,파도에 부딪혀 동글동글해진 몽돌들이 있는 해변,11,16.98,17.46


<br><hr>

### 04. 데이터프레임 저장

In [188]:
# 해수욕장 정보만
jeju_beach_info.to_csv('../data/jeju_beach/jeju_beach_info.csv', encoding='cp949', index=False)

In [189]:
# temp도 같이 있는 거
jeju_beach_info_with_temp.to_csv('../data/jeju_beach/jeju_beach_info_with_temp.csv', encoding='cp949', index=False)

In [190]:
12*22

264